In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_csv("mobiles.csv")
df.head(2)

screen_size  ROM  RAM  num_rear_camera  num_front_camera  battery_capacity   
0  Very Small   64    2                1                 1              1800  \
1       Small   64    4                2                 1              2815   

   ratings  num_of_ratings  sales_price  discount_percent   sales  
0      4.5           38645        32999              0.17  127.52  
1      4.5             244        57149              0.04    1.39

### Q1.

In [7]:
stat_mean = df["sales"].mean()
stat_std  = df["sales"].std()
stat_out  = stat_mean + 2 * stat_std
stat_out

146.55150129273218

In [8]:
df_q1 = df.loc[df["sales"] > stat_out, ].reset_index(drop = True)
len(df), len(df_q1)

(430, 16)

In [9]:
df_q1["idx"] = (df_q1["ROM"] / 32) + (df_q1["RAM"] / 2) + \
(df_q1["num_front_camera"] + df_q1["num_rear_camera"]) + \
(df_q1["battery_capacity"] / 1000)

In [10]:
df_q1.head(2)

screen_size  ROM  RAM  num_rear_camera  num_front_camera  battery_capacity   
0      Medium  128    6                2                 1              4000  \
1       Large  128    6                4                 2              4500   

   ratings  num_of_ratings  sales_price  discount_percent   sales   idx  
0      4.6          122001        18999              0.09  231.79  14.0  
1      4.5          267028        15999              0.20  427.22  17.5

In [12]:
round(df_q1["idx"].mean(), 2)

11.01

### Q2.

In [16]:
df_q2 = df.loc[df["num_rear_camera"] != 1, "battery_capacity":]
df_q2.head(2)

battery_capacity  ratings  num_of_ratings  sales_price  discount_percent   
1              2815      4.5             244        57149              0.04  \
4              2815      4.6             745        69149              0.02   

   sales  
1   1.39  
4   5.15

In [19]:
df_q2.corr()["sales"].abs().round(2)

battery_capacity    0.03
ratings             0.23
num_of_ratings      0.95
sales_price         0.25
discount_percent    0.22
sales               1.00
Name: sales, dtype: float64

In [20]:
df_q2.corr().round(2)

battery_capacity  ratings  num_of_ratings  sales_price   
battery_capacity              1.00    -0.42            0.03        -0.50  \
ratings                      -0.42     1.00            0.19         0.15   
num_of_ratings                0.03     0.19            1.00        -0.26   
sales_price                  -0.50     0.15           -0.26         1.00   
discount_percent              0.26    -0.12            0.21        -0.10   
sales                         0.03     0.23            0.95        -0.25   

                  discount_percent  sales  
battery_capacity              0.26   0.03  
ratings                      -0.12   0.23  
num_of_ratings                0.21   0.95  
sales_price                  -0.10  -0.25  
discount_percent              1.00   0.22  
sales                         0.22   1.00

In [21]:
df_q2.corr().reset_index()

index  battery_capacity   ratings  num_of_ratings  sales_price   
0  battery_capacity          1.000000 -0.424129        0.034902    -0.503019  \
1           ratings         -0.424129  1.000000        0.191655     0.151153   
2    num_of_ratings          0.034902  0.191655        1.000000    -0.260279   
3       sales_price         -0.503019  0.151153       -0.260279     1.000000   
4  discount_percent          0.257373 -0.118578        0.212442    -0.098640   
5             sales          0.025680  0.226075        0.949114    -0.247760   

   discount_percent     sales  
0          0.257373  0.025680  
1         -0.118578  0.226075  
2          0.212442  0.949114  
3         -0.098640 -0.247760  
4          1.000000  0.223471  
5          0.223471  1.000000

In [23]:
df_corr_melt = df_q2.corr().reset_index().melt(id_vars = "index")
df_corr_melt.head(2)

index          variable     value
0  battery_capacity  battery_capacity  1.000000
1           ratings  battery_capacity -0.424129

In [24]:
df_corr_melt.loc[df_corr_melt["index"] != df_corr_melt["variable"], ]

index          variable     value
1            ratings  battery_capacity -0.424129
2     num_of_ratings  battery_capacity  0.034902
3        sales_price  battery_capacity -0.503019
4   discount_percent  battery_capacity  0.257373
5              sales  battery_capacity  0.025680
6   battery_capacity           ratings -0.424129
8     num_of_ratings           ratings  0.191655
9        sales_price           ratings  0.151153
10  discount_percent           ratings -0.118578
11             sales           ratings  0.226075
12  battery_capacity    num_of_ratings  0.034902
13           ratings    num_of_ratings  0.191655
15       sales_price    num_of_ratings -0.260279
16  discount_percent    num_of_ratings  0.212442
17             sales    num_of_ratings  0.949114
18  battery_capacity       sales_price -0.503019
19           ratings       sales_price  0.151153
20    num_of_ratings       sales_price -0.260279
22  discount_percent       sales_price -0.098640
23             sales       sales_price -0.247760
24  battery_capacity  discount_percent  0.257373
25           ratings  discount_percent -0.118578
26    num_of_ratings  discount_percent  0.212442
27       sales_price  discount_percent -0.098640
29             sales  discount_percent  0.223471
30  battery_capacity             sales  0.025680
31           ratings             sales  0.226075
32    num_of_ratings             sales  0.949114
33       sales_price             sales -0.247760
34  discount_percent             sales  0.223471

### Q3.

In [25]:
df_q3 = df.copy()

In [31]:
# df_q3_dum = pd.get_dummies(df_q3, columns = ["screen_size"]) # 시험버전
df_q3_dum = pd.get_dummies(df_q3, columns = ["screen_size"],
                           dtype = "int") # 최신버전(2.0.0 기준)
df_q3_dum.head(2)

ROM  RAM  num_rear_camera  num_front_camera  battery_capacity  ratings   
0   64    2                1                 1              1800      4.5  \
1   64    4                2                 1              2815      4.5   

   num_of_ratings  sales_price  discount_percent   sales  screen_size_Large   
0           38645        32999              0.17  127.52                  0  \
1             244        57149              0.04    1.39                  0   

   screen_size_Medium  screen_size_Small  screen_size_Very Large   
0                   0                  0                       0  \
1                   0                  1                       0   

   screen_size_Very Small  
0                       1  
1                       0

In [40]:
df_q3_dum.shape

(430, 15)

In [39]:
df_q3_dum = df_q3_dum.set_index("sales").reset_index()
df_q3_dum.head(2)

In [35]:
df_train, df_test = train_test_split(df_q3_dum, train_size = 0.8, random_state = 123)
len(df_train), len(df_test)

(344, 86)

In [36]:
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

In [37]:
arr_train_nor[:2, ]

array([[0.00394753, 0.04761905, 0.18181818, 0.33333333, 0.        ,
        0.42307692, 0.625     , 0.00396262, 0.02607261, 0.09302326,
        0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.05214786, 0.23809524, 0.18181818, 0.        , 0.        ,
        0.21961538, 1.        , 0.01138668, 0.27392739, 0.18604651,
        0.        , 1.        , 0.        , 0.        , 0.        ]])

In [44]:
ls_k = [3, 5, 7, 9, 11]

k = ls_k[0]
model_knn = KNeighborsRegressor(n_neighbors = k)
model_knn.fit(X = arr_train_nor[:, 1:],
              y = arr_train_nor[:, 0])
pred = model_knn.predict(arr_test_nor[:, 1:])
val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0], 
                              y_pred = pred) ** 0.5
val_rmse

0.08186677375964535

In [46]:
ls_k = [3, 5, 7, 9, 11]

ls_rmse = []
for k in ls_k:
    model_knn = KNeighborsRegressor(n_neighbors = k)
    model_knn.fit(X = arr_train_nor[:, 1:],
                  y = arr_train_nor[:, 0])
    pred = model_knn.predict(arr_test_nor[:, 1:])
    val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0], 
                                  y_pred = pred) ** 0.5
    ls_rmse = ls_rmse + [val_rmse]

In [47]:
ls_rmse

[0.08186677375964535,
 0.09879109824384892,
 0.107669855645971,
 0.11232111394853059,
 0.1136902366621185]

In [48]:
pd.Series(ls_rmse, index = ls_k).idxmin()

3

In [52]:
df_train_nor = pd.DataFrame(arr_train_nor, columns = df_train.columns)
df_test_nor  = pd.DataFrame(arr_test_nor,  columns = df_test.columns)
df_train_nor["sales"][:4]

0    0.003948
1    0.052148
2    0.027957
3    0.039536
Name: sales, dtype: float64

### Q3. 추가 지시사항

다음은 저번달에 신규 출시된 경쟁사의 스마트폰 정보이다. 해당 스마트폰의 판매지수는 얼마로 예상되는가?  
※ 정규화 되지 않은 값으로 반올림하여 소수점 첫째 자리까지 출력하시오  
※ KNN 모델을 사용하며 이웃 개수는 직전에 최적이라고 판단한 k값을 사용하시오.  
* ROM: 256
* RAM: 6
* num_rear_camera: 4
* num_front_camera: 1
* battery_capacity: 4000
* ratings: 4.3
* num_of_ratings: 25000
* sales_price: 85000
* discount_percent: 0.05
* screen_size: "Large"

In [55]:
# df_s1 = pd.DataFrame(dict(ROM = 256, ~~~))
df_s1 = df_test.head(1).copy()
df_s1["RAM"] = 6
df_s1["num_rear_camera"] = 4
df_s1["battery_capacity"] = 4000
df_s1["ratings"] = 4.3
df_s1["num_of_ratings"] = 25000
df_s1["sales_price"] = 85000
df_s1["discount_percent"] = 0.05
df_s1["screen_size_Large"] = 1
df_s1["screen_size_Medium"] = 0
df_s1

sales  ROM  RAM  num_rear_camera  num_front_camera  battery_capacity   
13    5.9  256    6                4                 1              4000  \

    ratings  num_of_ratings  sales_price  discount_percent  screen_size_Large   
13      4.3           25000        85000              0.05                  1  \

    screen_size_Medium  screen_size_Small  screen_size_Very Large   
13                   0                  0                       0  \

    screen_size_Very Small  
13                       0

In [58]:
# arr_s1_nor = model_nor.transform(df_s1.drop(columns = "sales"))
arr_s1_nor = model_nor.transform(df_s1)
arr_s1_nor

array([[0.0119438 , 0.49206349, 0.45454545, 1.        , 0.        ,
        0.42307692, 0.625     , 0.05308122, 0.51815842, 0.09302326,
        1.        , 0.        , 0.        , 0.        , 0.        ]])

In [59]:
MinMaxScaler().fit_transform(df_s1) # 이렇게 하면 안됨!

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [83]:
model_knn_best = KNeighborsRegressor(n_neighbors = 3)
model_knn_best.fit(X = arr_train_nor[:, 1:],
                   y = arr_train_nor[:, 0])

KNeighborsRegressor(n_neighbors=3)

In [84]:
pred_s1 = model_knn_best.predict(arr_s1_nor[:, 1:])
pred_s1

array([0.00132259])

In [85]:
# model_nor.inverse_transform(pred_s1)
# model_nor.inverse_transform([pred_s1])
arr_s1_nor[0, 0] = pred_s1
arr_s1_inv = model_nor.inverse_transform(arr_s1_nor)
arr_s1_inv

array([[6.53333333e-01, 2.56000000e+02, 6.00000000e+00, 4.00000000e+00,
        1.00000000e+00, 4.00000000e+03, 4.30000000e+00, 2.50000000e+04,
        8.50000000e+04, 5.00000000e-02, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [86]:
df_s1_inv = pd.DataFrame(arr_s1_inv, columns = df_s1.columns)
df_s1_inv

sales    ROM  RAM  num_rear_camera  num_front_camera  battery_capacity   
0  0.653333  256.0  6.0              4.0               1.0            4000.0  \

   ratings  num_of_ratings  sales_price  discount_percent  screen_size_Large   
0      4.3         25000.0      85000.0              0.05                1.0  \

   screen_size_Medium  screen_size_Small  screen_size_Very Large   
0                 0.0                0.0                     0.0  \

   screen_size_Very Small  
0                     0.0

정답: 0.653333(0.7)

In [93]:
round(df_s1_inv.iloc[0, 0], 1)

0.7

In [87]:
df_s1

sales  ROM  RAM  num_rear_camera  num_front_camera  battery_capacity   
13    5.9  256    6                4                 1              4000  \

    ratings  num_of_ratings  sales_price  discount_percent  screen_size_Large   
13      4.3           25000        85000              0.05                  1  \

    screen_size_Medium  screen_size_Small  screen_size_Very Large   
13                   0                  0                       0  \

    screen_size_Very Small  
13                       0

In [88]:
df.head(2)

screen_size  ROM  RAM  num_rear_camera  num_front_camera  battery_capacity   
0  Very Small   64    2                1                 1              1800  \
1       Small   64    4                2                 1              2815   

   ratings  num_of_ratings  sales_price  discount_percent   sales  
0      4.5           38645        32999              0.17  127.52  
1      4.5             244        57149              0.04    1.39

In [89]:
df_sub = df.iloc[:, 1:7]
df_sub.head(2)

ROM  RAM  num_rear_camera  num_front_camera  battery_capacity  ratings
0   64    2                1                 1              1800      4.5
1   64    4                2                 1              2815      4.5

In [90]:
df_nors = df_sub.max().reset_index().merge(df_sub.min().reset_index(), on = "index")
df_nors.columns = ["cols", "max", "min"]
df_nors

cols     max     min
0               ROM   512.0     8.0
1               RAM    12.0     1.0
2   num_rear_camera     4.0     1.0
3  num_front_camera     3.0     1.0
4  battery_capacity  7000.0  1800.0
5           ratings     4.6     3.0

In [91]:
def udf_nor_mm(x):
    return (x - x.min()) / (x.max() - x.min())

In [92]:
df_sub_nor = df_sub.apply(udf_nor_mm)
df_sub_nor.head(2)

ROM       RAM  num_rear_camera  num_front_camera  battery_capacity   
0  0.111111  0.090909         0.000000               0.0          0.000000  \
1  0.111111  0.272727         0.333333               0.0          0.195192   

   ratings  
0   0.9375  
1   0.9375